In [1]:
import numpy as np
from bitstring import Bits
from bitstring import BitArray
import pickle
import cv2
import time

In [2]:
def convert_binary_number_to_2complement_binary_number(number, bit_width):
	temp_number = number[-bit_width:]

	binary_number = ''.join(['1' if i == '0' else '0' for i in temp_number])
	last_bit = 1

	for i in range(bit_width):
		if(last_bit == 1):
			if(binary_number[bit_width - 1 - i] == '1'):
				binary_number = list(binary_number)
				binary_number[bit_width - 1 - i] = '0'
				binary_number = ''.join(binary_number)
				last_bit = 1
			else:
				binary_number = list(binary_number)
				binary_number[bit_width - 1 - i] = '1'
				binary_number = ''.join(binary_number)
				last_bit = 0

	return '0b' + binary_number

def convert_pos_int_to_binary(pos_number, bit_width):
	if((bit_width%4) != 0):
		return str(Bits(int=pos_number, length=bit_width))
	else:
		temp =  str(Bits(int=pos_number, length=bit_width+1))
		return '0b' + temp[-(len(temp)-3):]

def convert_pos_fraction_to_binary(pos_number, bit_width):
	binary_number = ''
	temp_number = pos_number

	for i in range(bit_width):
		x2_temp_number = temp_number*2

		if(int(x2_temp_number) == 1):
			binary_number = binary_number + '1'
			temp_number = x2_temp_number - 1
		else:
			binary_number = binary_number + '0'
			temp_number = x2_temp_number

	return binary_number

def convert_pos_fixedpoint_to_binary(pos_number, bit_width):
	pos_integer = int(pos_number)
	pos_fraction = pos_number - pos_integer

	bit_width_for_pos_integer = int(bit_width/2)
	bit_width_for_fraction = int(bit_width/2)

	pos_integer_binary_number = convert_pos_int_to_binary(pos_integer, bit_width_for_pos_integer)
	pos_fraction_binary_number = convert_pos_fraction_to_binary(pos_fraction, bit_width_for_fraction)

	return pos_integer_binary_number + pos_fraction_binary_number

def convert_neg_fixedpoint_to_binary(neg_number, bit_width):
	pos_integer = int(0 - neg_number)
	pos_fraction = 0 - neg_number - pos_integer

	bit_width_for_integer = int(bit_width/2)
	bit_width_for_fraction = int(bit_width/2)

	pos_integer_binary_number = convert_pos_int_to_binary(pos_integer, bit_width_for_integer)
	pos_fraction_binary_number = convert_pos_fraction_to_binary(pos_fraction, bit_width_for_fraction)

	pos_binary_number = pos_integer_binary_number + pos_fraction_binary_number
	neg_binary_number = convert_binary_number_to_2complement_binary_number(pos_binary_number, bit_width)

	return neg_binary_number
	
def convert_binary_to_int(number):
	integer = BitArray(bin=number)
	integer = integer.int
	return integer

def convert_binary_to_uint(number):
	unsigned_integer = BitArray(bin=number)
	unsigned_integer = unsigned_integer.uint
	return unsigned_integer

def convert_binary_to_pos_float_fraction(number, bit_width):
	fraction = 0
	for i in range(bit_width):
		if(number[i] == '1'):
			fraction = fraction + 2**(- (i + 1))

	return fraction

def convert_binary_to_pos_fixedpoint(pos_number, bit_width):
	bit_width_for_integer = int(bit_width/2)
	bit_width_for_fraction = int(bit_width/2)

	pos_integer = pos_number[0:bit_width_for_integer + 2]
	pos_fraction = pos_number[-bit_width_for_fraction:]

	pos_integer = BitArray(bin=pos_integer)
	pos_integer = pos_integer.uint

	pos_fraction = convert_binary_to_pos_float_fraction(pos_fraction, bit_width_for_fraction)

	pos_fixedpoint = pos_integer + pos_fraction

	return pos_fixedpoint

def convert_binary_to_neg_fixedpoint(neg_number, bit_width):
	pos_number = convert_binary_number_to_2complement_binary_number(neg_number, bit_width)

	bit_width_for_integer = int(bit_width/2)
	bit_width_for_fraction = int(bit_width/2)

	pos_integer = pos_number[0:bit_width_for_integer + 2]
	pos_fraction = pos_number[-bit_width_for_fraction:]

	pos_integer = BitArray(bin=pos_integer)
	pos_integer = pos_integer.uint

	pos_fraction = convert_binary_to_pos_float_fraction(pos_fraction, bit_width_for_fraction)

	pos_fixedpoint = pos_integer + pos_fraction

	return 0 - pos_fixedpoint
	

In [3]:
max_image = 1
min_image = -1

max_filter_conv0 = 0.64
min_filter_conv0 = -0.64
max_filter_conv1 = 0.94
min_filter_conv1 = -0.94

max_result_conv0 = 2.67
min_result_conv0 = -2.67
max_result_conv1 = 8.88
min_result_conv1 = -8.88

max_bias_conv0 = 0.35
min_bias_conv0 = -0.35
max_bias_conv1 = 0.16
min_bias_conv1 = -0.16

max_pooling0 = 1.75
min_pooling0 = -1.75
max_pooling1 = 2.84
min_pooling1 = -2.84

max_w0 = 1.02
min_w0 = -1.02
max_w1 = 0.78
min_w1 = -0.78
max_w2 = 0.85
min_w2 = -0.85

max_b0 = 0.21
min_b0 = -0.21
max_b1 = 0.19
min_b1 = -0.19
max_b2 = 0.24
min_b2 = -0.24

max_in = 2.84
min_in = -2.84
max_hd0 = 9.6
min_hd0 = -9.6
max_hd1 = 12.8
min_hd1 = -12.8
max_out_bf_act = 23.4
min_out_bf_act = -23.4
max_out = 1
min_out = -1

In [4]:
# ============================================ Bit-width ============================================
bit_width_image = 6
bit_width_filter_conv0 = 6
bit_width_filter_conv1 = 6
bit_width_conv0 = 6
bit_width_conv1 = 6
bit_width_bias_conv0 = 6
bit_width_bias_conv1 = 6
bit_width_pooling0 = 6
bit_width_pooling1 = 6
bit_width_input = bit_width_pooling1

bit_width_w0 = 6
bit_width_w1 = 6
bit_width_w2 = 6

bit_width_b0 = 6
bit_width_b1 = 6
bit_width_b2 = 6

bit_width_h0 = 6
bit_width_h1 = 6
bit_width_bf_sigmoid = 6
bit_width_output = 6

In [5]:
# ============================================ Max-min quantization value ============================================
max_image_q = 2**(bit_width_image-1)-1
min_image_q = -2**(bit_width_image-1)
max_filter_conv0_q = 2**(bit_width_filter_conv0-1)-1
min_filter_conv0_q = -2**(bit_width_filter_conv0-1)
max_filter_conv1_q = 2**(bit_width_filter_conv1-1)-1
min_filter_conv1_q = -2**(bit_width_filter_conv1-1)
max_conv0_q = 2**(bit_width_conv0-1)-1
min_conv0_q = -2**(bit_width_conv0-1)
max_conv1_q = 2**(bit_width_conv1-1)-1
min_conv1_q = -2**(bit_width_conv1-1)
max_bias_conv0_q = 2**(bit_width_bias_conv0-1)-1
min_bias_conv0_q = -2**(bit_width_bias_conv0-1)
max_bias_conv1_q = 2**(bit_width_bias_conv1-1)-1
min_bias_conv1_q = -2**(bit_width_bias_conv1-1)
max_pooling0_q = 2**(bit_width_pooling0-1)-1
min_pooling0_q = -2**(bit_width_pooling0-1)
max_pooling1_q = 2**(bit_width_pooling1-1)-1
min_pooling1_q = -2**(bit_width_pooling1-1)

max_w0_q = 2**(bit_width_w0-1)-1
min_w0_q = -2**(bit_width_w0-1)
max_w1_q = 2**(bit_width_w1-1)-1
min_w1_q = -2**(bit_width_w1-1)
max_w2_q = 2**(bit_width_w2-1)-1
min_w2_q = -2**(bit_width_w2-1)

max_b0_q = 2**(bit_width_b0-1)-1
min_b0_q = -2**(bit_width_b0-1)
max_b1_q = 2**(bit_width_b1-1)-1
min_b1_q = -2**(bit_width_b1-1)
max_b2_q = 2**(bit_width_b2-1)-1
min_b2_q = -2**(bit_width_b2-1)

max_input_q = 2**(bit_width_input-1)-1
min_input_q = -2**(bit_width_input-1)
max_hd0_q = 2**(bit_width_h0-1)-1
min_hd0_q = -2**(bit_width_h0-1)
max_hd1_q = 2**(bit_width_h1-1)-1
min_hd1_q = -2**(bit_width_h1-1)
max_bf_sigmoid_q = 2**(bit_width_bf_sigmoid-1)-1
min_bf_sigmoid_q = -2**(bit_width_bf_sigmoid-1)
max_output_q = 2**(bit_width_output-1)-1
min_output_q = -2**(bit_width_output-1)

In [6]:
# ============================================ Scale and offset for w1 & bias ============================================
# Quantied number = (X/Scale) + Offset
scale_image = (max_image - min_image)/(max_image_q - min_image_q)
scale_filter_conv0 = (max_filter_conv0 - min_filter_conv0)/(max_filter_conv0_q - min_filter_conv0_q)
scale_filter_conv1 = (max_filter_conv1 - min_filter_conv1)/(max_filter_conv1_q - min_filter_conv1_q)
scale_conv0 = (max_result_conv0 - min_result_conv0)/(max_conv0_q - min_conv0_q)
scale_conv1 = (max_result_conv1 - min_result_conv1)/(max_conv1_q - min_conv1_q)
scale_bias_conv0 = (max_bias_conv0 - min_bias_conv0)/(max_bias_conv0_q - min_bias_conv0_q)
scale_bias_conv1 = (max_bias_conv1 - min_bias_conv1)/(max_bias_conv1_q - min_bias_conv1_q)
scale_pooling0 = (max_pooling0 - min_pooling0)/(max_pooling0_q - min_pooling0_q)
scale_pooling1 = (max_pooling1 - min_pooling1)/(max_pooling1_q - min_pooling1_q)

scale_w0 = (max_w0 - min_w0)/(max_w0_q - min_w0_q)
scale_w1 = (max_w1 - min_w1)/(max_w1_q - min_w1_q)
scale_w2 = (max_w2 - min_w2)/(max_w2_q - min_w2_q)

scale_b0 = (max_b0 - min_b0)/(max_b0_q - min_b0_q)
scale_b1 = (max_b1 - min_b1)/(max_b1_q - min_b1_q)
scale_b2 = (max_b2 - min_b2)/(max_b2_q - min_b2_q)

scale_input = (max_in - min_in)/(max_input_q - min_input_q)
scale_hd0 = (max_hd0 - min_hd0)/(max_hd0_q - min_hd0_q)
scale_hd1 = (max_hd1 - min_hd1)/(max_hd1_q - min_hd1_q)
scale_bf_sigmoid = (max_out_bf_act - min_out_bf_act)/(max_bf_sigmoid_q - min_bf_sigmoid_q)
scale_output = (max_out - min_out)/(max_output_q - min_output_q)

offset_image = round((max_image*min_image_q - min_image*max_image_q)/(max_image - min_image))
offset_filter_conv0 = round((max_filter_conv0*min_filter_conv0_q - min_filter_conv0*max_filter_conv0_q)/(max_filter_conv0 - min_filter_conv0))
offset_filter_conv1 = round((max_filter_conv1*min_filter_conv1_q - min_filter_conv1*max_filter_conv1_q)/(max_filter_conv1 - min_filter_conv1))
offset_conv0 = round((max_result_conv0*min_conv0_q - min_result_conv0*max_conv0_q)/(max_result_conv0 - min_result_conv0))
offset_conv1 = round((max_result_conv1*min_conv1_q - min_result_conv1*max_conv1_q)/(max_result_conv1 - min_result_conv1))
offset_bias_conv0 = round((max_bias_conv0*min_bias_conv0_q - min_bias_conv0*max_bias_conv0_q)/(max_bias_conv0 - min_bias_conv0))
offset_bias_conv1 = round((max_bias_conv1*min_bias_conv1_q - min_bias_conv1*max_bias_conv1_q)/(max_bias_conv1 - min_bias_conv1))
offset_pooling0 = round((max_pooling0*min_pooling0_q - min_pooling0*max_pooling0_q)/(max_pooling0 - min_pooling0))
offset_pooling1 = round((max_pooling1*min_pooling1_q - min_pooling1*max_pooling1_q)/(max_pooling1 - min_pooling1))

offset_w0 = round((max_w0*min_w0_q - min_w0*max_w0_q)/(max_w0 - min_w0))
offset_w1 = round((max_w1*min_w1_q - min_w1*max_w1_q)/(max_w1 - min_w1))
offset_w2 = round((max_w2*min_w2_q - min_w2*max_w2_q)/(max_w2 - min_w2))

offset_b0 = round((max_b0*min_b0_q - min_b0*max_b0_q)/(max_b0 - min_b0))
offset_b1 = round((max_b1*min_b1_q - min_b1*max_b1_q)/(max_b1 - min_b1))
offset_b2 = round((max_b2*min_b2_q - min_b2*max_b2_q)/(max_b2 - min_b2))

offset_input = round((max_in*min_input_q - min_in*max_input_q)/(max_in - min_in))
offset_hd0 = round((max_hd0*min_hd0_q - min_hd0*max_hd0_q)/(max_hd0 - min_hd0))
offset_hd1 = round((max_hd1*min_hd1_q - min_hd1*max_hd1_q)/(max_hd1 - min_hd1))
offset_bf_sigmoid = round((max_out_bf_act*min_bf_sigmoid_q - min_out_bf_act*max_bf_sigmoid_q)/(max_out_bf_act - min_out_bf_act))
offset_output = round((max_out*min_output_q - min_out*max_output_q)/(max_out - min_out))

# ============================================ Scale for quantize ============================================
# X * Scale 
scale_calculate_convolution_0 = (scale_image*scale_filter_conv0)/scale_conv0
scale_for_pooling0 = scale_conv0/scale_pooling0
scale_calculate_convolution_1 = (scale_pooling0*scale_filter_conv1)/scale_conv1
scale_for_pooling1 = scale_conv1/scale_pooling1

scale_for_W0_x_Flatten = (scale_input*scale_w0)/scale_hd0
scale_for_W1_x_hd0 = (scale_hd0*scale_w1)/scale_hd1
scale_for_W2_x_hd1 = (scale_hd1*scale_w2)/scale_bf_sigmoid

scale_for_B0_conv = scale_bias_conv0/scale_conv0
scale_for_B1_conv = scale_bias_conv1/scale_conv1

scale_for_B0_fc = scale_b0/scale_hd0
scale_for_B1_fc = scale_b1/scale_hd1
scale_for_B2_fc = scale_b2/scale_bf_sigmoid

scale_quant_bf_sigmoid_layer = 1/scale_bf_sigmoid
scale_dequant_bf_sigmoid_layer = scale_bf_sigmoid

max_sigmoid_layer = max_out
min_sigmoid_layer = min_out
scale_quant_sigmoid_layer = 1/scale_output
scale_dequant_sigmoid_layer = scale_output


In [7]:
fixedpoint_bit_width = 34

In [8]:
# ========================== BINARY NUMBER ==========================
scale_w0_bin = convert_pos_fixedpoint_to_binary(scale_w0, fixedpoint_bit_width)
scale_w1_bin = convert_pos_fixedpoint_to_binary(scale_w1, fixedpoint_bit_width)
scale_w2_bin = convert_pos_fixedpoint_to_binary(scale_w2, fixedpoint_bit_width)

scale_b0_bin = convert_pos_fixedpoint_to_binary(scale_b0, fixedpoint_bit_width)
scale_b1_bin = convert_pos_fixedpoint_to_binary(scale_b1, fixedpoint_bit_width)
scale_b2_bin = convert_pos_fixedpoint_to_binary(scale_b2, fixedpoint_bit_width)

scale_input_bin = convert_pos_fixedpoint_to_binary(scale_input, fixedpoint_bit_width)
scale_layer_0_bin = convert_pos_fixedpoint_to_binary(scale_hd0, fixedpoint_bit_width)
scale_layer_1_bin = convert_pos_fixedpoint_to_binary(scale_hd1, fixedpoint_bit_width)
scale_bf_sigmoid_bin = convert_pos_fixedpoint_to_binary(scale_bf_sigmoid, fixedpoint_bit_width)
scale_output_bin = convert_pos_fixedpoint_to_binary(scale_output, fixedpoint_bit_width)

scale_calculate_convolution_0_bin = convert_pos_fixedpoint_to_binary(scale_calculate_convolution_0, fixedpoint_bit_width)
scale_for_pooling0_bin = convert_pos_fixedpoint_to_binary(scale_for_pooling0, fixedpoint_bit_width)
scale_calculate_convolution_1_bin = convert_pos_fixedpoint_to_binary(scale_calculate_convolution_1, fixedpoint_bit_width)
scale_for_pooling1_bin = convert_pos_fixedpoint_to_binary(scale_for_pooling1, fixedpoint_bit_width)

scale_for_W0_x_Flatten_bin = convert_pos_fixedpoint_to_binary(scale_for_W0_x_Flatten, fixedpoint_bit_width)
scale_for_W1_x_hd0_bin = convert_pos_fixedpoint_to_binary(scale_for_W1_x_hd0, fixedpoint_bit_width)
scale_for_W2_x_hd1_bin = convert_pos_fixedpoint_to_binary(scale_for_W2_x_hd1, fixedpoint_bit_width)

scale_for_B0_conv_bin = convert_pos_fixedpoint_to_binary(scale_for_B0_conv, fixedpoint_bit_width)
scale_for_B1_conv_bin = convert_pos_fixedpoint_to_binary(scale_for_B1_conv, fixedpoint_bit_width)

scale_for_B0_fc_bin = convert_pos_fixedpoint_to_binary(scale_for_B0_fc, fixedpoint_bit_width)
scale_for_B1_fc_bin = convert_pos_fixedpoint_to_binary(scale_for_B1_fc, fixedpoint_bit_width)
scale_for_B2_fc_bin = convert_pos_fixedpoint_to_binary(scale_for_B2_fc, fixedpoint_bit_width)

scale_quant_bf_sigmoid_layer_bin = convert_pos_fixedpoint_to_binary(scale_quant_bf_sigmoid_layer, fixedpoint_bit_width)
scale_dequant_bf_sigmoid_layer_bin = convert_pos_fixedpoint_to_binary(scale_dequant_bf_sigmoid_layer, fixedpoint_bit_width)

max_sigmoid_layer_bin = convert_pos_fixedpoint_to_binary(max_sigmoid_layer, fixedpoint_bit_width)
min_sigmoid_layer_bin = convert_pos_fixedpoint_to_binary(min_sigmoid_layer, fixedpoint_bit_width)
scale_quant_sigmoid_layer_bin = convert_pos_fixedpoint_to_binary(scale_quant_sigmoid_layer, fixedpoint_bit_width)
scale_dequant_sigmoid_layer_bin = convert_pos_fixedpoint_to_binary(scale_dequant_sigmoid_layer, fixedpoint_bit_width)

# ============================================================================================================================================================

scale_w0_bin_to_int = convert_binary_to_uint(scale_w0_bin)
scale_w1_bin_to_int = convert_binary_to_uint(scale_w1_bin)
scale_w2_bin_to_int = convert_binary_to_uint(scale_w2_bin)

scale_b0_bin_to_int = convert_binary_to_uint(scale_b0_bin)
scale_b1_bin_to_int = convert_binary_to_uint(scale_b1_bin)
scale_b2_bin_to_int = convert_binary_to_uint(scale_b2_bin)

scale_input_bin_to_int = convert_binary_to_uint(scale_input_bin)
scale_layer_0_bin_to_int = convert_binary_to_uint(scale_layer_0_bin)
scale_layer_1_bin_to_int = convert_binary_to_uint(scale_layer_1_bin)
scale_bf_sigmoid_bin_to_int = convert_binary_to_uint(scale_bf_sigmoid_bin)
scale_output_bin_to_int = convert_binary_to_uint(scale_output_bin)

scale_calculate_convolution_0_bin_to_int = convert_binary_to_uint(scale_calculate_convolution_0_bin)
scale_for_pooling0_bin_to_int = convert_binary_to_uint(scale_for_pooling0_bin)
scale_calculate_convolution_1_bin_to_int = convert_binary_to_uint(scale_calculate_convolution_1_bin)
scale_for_pooling1_bin_to_int = convert_binary_to_uint(scale_for_pooling1_bin)

scale_for_W0_x_Flatten_bin_to_int = convert_binary_to_uint(scale_for_W0_x_Flatten_bin)
scale_for_W1_x_hd0_bin_to_int = convert_binary_to_uint(scale_for_W1_x_hd0_bin)
scale_for_W2_x_hd1_bin_to_int = convert_binary_to_uint(scale_for_W2_x_hd1_bin)

scale_for_B0_conv_bin_to_int = convert_binary_to_uint(scale_for_B0_conv_bin)
scale_for_B1_conv_bin_to_int = convert_binary_to_uint(scale_for_B1_conv_bin)

scale_for_B0_fc_bin_to_int = convert_binary_to_uint(scale_for_B0_fc_bin)
scale_for_B1_fc_bin_to_int = convert_binary_to_uint(scale_for_B1_fc_bin)
scale_for_B2_fc_bin_to_int = convert_binary_to_uint(scale_for_B2_fc_bin)

scale_quant_bf_sigmoid_layer_bin_to_int = convert_binary_to_uint(scale_quant_bf_sigmoid_layer_bin)
scale_dequant_bf_sigmoid_layer_bin_to_int = convert_binary_to_uint(scale_dequant_bf_sigmoid_layer_bin)

max_sigmoid_layer_bin_to_int = convert_binary_to_uint(max_sigmoid_layer_bin)
min_sigmoid_layer_bin_to_int = convert_binary_to_uint(min_sigmoid_layer_bin)
scale_quant_sigmoid_layer_bin_to_int = convert_binary_to_uint(scale_quant_sigmoid_layer_bin)
scale_dequant_sigmoid_layer_bin_to_int = convert_binary_to_uint(scale_dequant_sigmoid_layer_bin)

# ============================================================================================================================================================

scale_w0_for_check_python = convert_binary_to_pos_fixedpoint(scale_w0_bin, fixedpoint_bit_width)
scale_w1_for_check_python = convert_binary_to_pos_fixedpoint(scale_w1_bin, fixedpoint_bit_width)
scale_w2_for_check_python = convert_binary_to_pos_fixedpoint(scale_w2_bin, fixedpoint_bit_width)

scale_b0_for_check_python = convert_binary_to_pos_fixedpoint(scale_b0_bin, fixedpoint_bit_width)
scale_b1_for_check_python = convert_binary_to_pos_fixedpoint(scale_b1_bin, fixedpoint_bit_width)
scale_b2_for_check_python = convert_binary_to_pos_fixedpoint(scale_b2_bin, fixedpoint_bit_width)

scale_input_int_for_check_python = convert_binary_to_pos_fixedpoint(scale_input_bin, fixedpoint_bit_width)
scale_layer_0_int_for_check_python = convert_binary_to_pos_fixedpoint(scale_layer_0_bin, fixedpoint_bit_width)
scale_layer_1_int_for_check_python = convert_binary_to_pos_fixedpoint(scale_layer_1_bin, fixedpoint_bit_width)
scale_bf_sigmoid_int_for_check_python = convert_binary_to_pos_fixedpoint(scale_bf_sigmoid_bin, fixedpoint_bit_width)
scale_output_int_for_check_python = convert_binary_to_pos_fixedpoint(scale_output_bin, fixedpoint_bit_width)

scale_calculate_convolution_0_for_check_python = convert_binary_to_pos_fixedpoint(scale_calculate_convolution_0_bin, fixedpoint_bit_width)
scale_for_pooling0_for_check_python = convert_binary_to_pos_fixedpoint(scale_for_pooling0_bin, fixedpoint_bit_width)
scale_calculate_convolution_1_for_check_pythont = convert_binary_to_pos_fixedpoint(scale_calculate_convolution_1_bin, fixedpoint_bit_width)
scale_for_pooling1_for_check_python = convert_binary_to_pos_fixedpoint(scale_for_pooling1_bin, fixedpoint_bit_width)

scale_for_W0_x_Flatten_for_check_python = convert_binary_to_pos_fixedpoint(scale_for_W0_x_Flatten_bin, fixedpoint_bit_width)
scale_for_W1_x_hd0_for_check_python = convert_binary_to_pos_fixedpoint(scale_for_W1_x_hd0_bin, fixedpoint_bit_width)
scale_for_W2_x_hd1_for_check_python = convert_binary_to_pos_fixedpoint(scale_for_W2_x_hd1_bin, fixedpoint_bit_width)

scale_for_B0_conv_for_check_python = convert_binary_to_pos_fixedpoint(scale_for_B0_conv_bin, fixedpoint_bit_width)
scale_for_B1_conv_for_check_python = convert_binary_to_pos_fixedpoint(scale_for_B1_conv_bin, fixedpoint_bit_width)

scale_for_B0_fc_for_check_python = convert_binary_to_pos_fixedpoint(scale_for_B0_fc_bin, fixedpoint_bit_width)
scale_for_B1_fc_for_check_python = convert_binary_to_pos_fixedpoint(scale_for_B1_fc_bin, fixedpoint_bit_width)
scale_for_B2_fc_for_check_python = convert_binary_to_pos_fixedpoint(scale_for_B2_fc_bin, fixedpoint_bit_width)

scale_quant_bf_sigmoid_layer_for_check_python = convert_binary_to_pos_fixedpoint(scale_quant_bf_sigmoid_layer_bin, fixedpoint_bit_width)
scale_dequant_bf_sigmoid_layer_for_check_python = convert_binary_to_pos_fixedpoint(scale_dequant_bf_sigmoid_layer_bin, fixedpoint_bit_width)

max_sigmoid_layer_for_check_python = convert_binary_to_pos_fixedpoint(max_sigmoid_layer_bin, fixedpoint_bit_width)
min_sigmoid_layer_for_check_python = convert_binary_to_neg_fixedpoint(min_sigmoid_layer_bin, fixedpoint_bit_width)
scale_quant_sigmoid_layer_for_check_python = convert_binary_to_pos_fixedpoint(scale_quant_sigmoid_layer_bin, fixedpoint_bit_width)
scale_dequant_sigmoid_layer_for_check_python = convert_binary_to_pos_fixedpoint(scale_dequant_sigmoid_layer_bin, fixedpoint_bit_width)

In [9]:
print('scale_w0', scale_w0)
print('scale_w1', scale_w1)
print('scale_w2', scale_w2)

print('\n')

print('scale_b0', scale_b0)
print('scale_b1', scale_b1)
print('scale_b2', scale_b2)

print('\n')

print('scale_input', scale_input)
print('scale_layer_0', scale_hd0)
print('scale_layer_1', scale_hd0)
print('scale_bf_sigmoid', scale_bf_sigmoid)
print('scale_output', scale_output)

print('\n')

print('scale_calculate_convolution_0', scale_calculate_convolution_0)
print('scale_for_pooling0', scale_for_pooling0)
print('scale_calculate_convolution_1', scale_calculate_convolution_1)
print('scale_for_pooling1', scale_for_pooling1)

print('\n')

print('scale_for_W0_x_Flatten', scale_for_W0_x_Flatten)
print('scale_for_W1_x_hd0', scale_for_W1_x_hd0)
print('scale_for_W2_x_hd1', scale_for_W2_x_hd1)

print('\n')

print('scale_for_B0_conv', scale_for_B0_conv)
print('scale_for_B1_conv', scale_for_B1_conv)

print('\n')

print('scale_for_B0_fc', scale_for_B0_fc)
print('scale_for_B1_fc', scale_for_B1_fc)
print('scale_for_B2_fc', scale_for_B2_fc)

print('\n')

print('scale_quant_bf_sigmoid_layer', scale_quant_bf_sigmoid_layer)
print('scale_dequant_bf_sigmoid_layer', scale_dequant_bf_sigmoid_layer)

print('\n')

print('max_sigmoid_layer', max_sigmoid_layer)
print('min_sigmoid_layer', min_sigmoid_layer)
print('scale_quant_sigmoid_layer', scale_quant_sigmoid_layer)
print('scale_dequant_sigmoid_layer', scale_dequant_sigmoid_layer)

print('\n#################################################################\n')

print('scale_w0_bin', scale_w0_bin)
print('scale_w1_bin', scale_w1_bin)
print('scale_w2_bin', scale_w2_bin)

print('\n')

print('scale_b0_bin', scale_b0_bin)
print('scale_b1_bin', scale_b1_bin)
print('scale_b2_bin', scale_b2_bin)

print('\n')

print('scale_input_bin', scale_input_bin)
print('scale_layer_0_bin', scale_layer_0_bin)
print('scale_layer_1_bin', scale_layer_1_bin)
print('scale_bf_sigmoid_bin', scale_bf_sigmoid_bin)
print('scale_output_bin', scale_output_bin)

print('\n')

print('scale_calculate_convolution_0_bin', scale_calculate_convolution_0_bin)
print('scale_for_pooling0_bin', scale_for_pooling0_bin)
print('scale_calculate_convolution_1_bin', scale_calculate_convolution_1_bin)
print('scale_for_pooling1_bin', scale_for_pooling1_bin)

print('\n')

print('scale_for_W0_x_Flatten_bin', scale_for_W0_x_Flatten_bin)
print('scale_for_W1_x_hd0_bin', scale_for_W1_x_hd0_bin)
print('scale_for_W2_x_hd1_bin', scale_for_W2_x_hd1_bin)

print('\n')

print('scale_for_B0_conv_bin', scale_for_B0_conv_bin)
print('scale_for_B1_conv_bin', scale_for_B1_conv_bin)

print('\n')

print('scale_for_B0_fc_bin', scale_for_B0_fc_bin)
print('scale_for_B1_fc_bin', scale_for_B1_fc_bin)
print('scale_for_B2_fc_bin', scale_for_B2_fc_bin)

print('\n')

print('scale_quant_bf_sigmoid_layer_bin', scale_quant_bf_sigmoid_layer_bin)
print('scale_dequant_bf_sigmoid_layer_bin', scale_dequant_bf_sigmoid_layer_bin)

print('\n')

print('max_sigmoid_layer_bin', max_sigmoid_layer_bin)
print('min_sigmoid_layer_bin', min_sigmoid_layer_bin)
print('scale_quant_sigmoid_layer_bin', scale_quant_sigmoid_layer_bin)
print('scale_dequant_sigmoid_layer_bin', scale_dequant_sigmoid_layer_bin)

print('\n#################################################################\n')

print('scale_w0_bin_to_int', scale_w0_bin_to_int)
print('scale_w1_bin_to_int', scale_w1_bin_to_int)
print('scale_w2_bin_to_int', scale_w2_bin_to_int)

print('\n')

print('scale_b0_bin_to_int', scale_b0_bin_to_int)
print('scale_b1_bin_to_int', scale_b1_bin_to_int)
print('scale_b2_bin_to_int', scale_b2_bin_to_int)

print('\n')

print('scale_input_bin_to_int', scale_input_bin_to_int)
print('scale_layer_0_bin_to_int', scale_layer_0_bin_to_int)
print('scale_layer_1_bin_to_int', scale_layer_1_bin_to_int)
print('scale_bf_sigmoid_bin_to_int', scale_bf_sigmoid_bin_to_int)
print('scale_output_bin_to_int', scale_output_bin_to_int)

print('\n')

print('scale_calculate_convolution_0_bin_to_int', scale_calculate_convolution_0_bin_to_int)
print('scale_for_pooling0_bin_to_int', scale_for_pooling0_bin_to_int)
print('scale_calculate_convolution_1_bin_to_int', scale_calculate_convolution_1_bin_to_int)
print('scale_for_pooling1_bin_to_int', scale_for_pooling1_bin_to_int)

print('\n')

print('scale_for_W0_x_Flatten_bin_to_int', scale_for_W0_x_Flatten_bin_to_int)
print('scale_for_W1_x_hd0_bin_to_int', scale_for_W1_x_hd0_bin_to_int)
print('scale_for_W2_x_hd1_bin_to_int', scale_for_W2_x_hd1_bin_to_int)

print('\n')

print('scale_for_B0_conv_bin_to_int', scale_for_B0_conv_bin_to_int)
print('scale_for_B1_conv_bin_to_int', scale_for_B1_conv_bin_to_int)

print('\n')

print('scale_for_B0_fc_bin_to_int', scale_for_B0_fc_bin_to_int)
print('scale_for_B1_fc_bin_to_int', scale_for_B1_fc_bin_to_int)
print('scale_for_B2_fc_bin_to_int', scale_for_B2_fc_bin_to_int)

print('\n')

print('scale_quant_bf_sigmoid_layer_bin_to_int', scale_quant_bf_sigmoid_layer_bin_to_int)
print('scale_dequant_bf_sigmoid_layer_bin_to_int', scale_dequant_bf_sigmoid_layer_bin_to_int)

print('\n')

print('max_sigmoid_layer_bin_to_int', max_sigmoid_layer_bin_to_int)
print('min_sigmoid_layer_bin_to_int', min_sigmoid_layer_bin_to_int)
print('scale_quant_sigmoid_layer_bin_to_int', scale_quant_sigmoid_layer_bin_to_int)
print('scale_dequant_sigmoid_layer_bin_to_int', scale_dequant_sigmoid_layer_bin_to_int)

print('\n#################################################################\n')

print('scale_w0_for_check_python', scale_w0_for_check_python)
print('scale_w1_for_check_python', scale_w1_for_check_python)
print('scale_w2_for_check_python', scale_w2_for_check_python)

print('\n')

print('scale_b0_for_check_python', scale_b0_for_check_python)
print('scale_b1_for_check_python', scale_b1_for_check_python)
print('scale_b2_for_check_python', scale_b2_for_check_python)

print('\n')

print('scale_input_int_for_check_python', scale_input_int_for_check_python)
print('scale_layer_0_int_for_check_python', scale_layer_0_int_for_check_python)
print('scale_layer_1_int_for_check_python', scale_layer_1_int_for_check_python)
print('scale_bf_sigmoid_int_for_check_python', scale_bf_sigmoid_int_for_check_python)
print('scale_output_int_for_check_python', scale_output_int_for_check_python)

print('\n')

print('scale_calculate_convolution_0', scale_calculate_convolution_0)
print('scale_for_pooling0', scale_for_pooling0)
print('scale_calculate_convolution_1', scale_calculate_convolution_1)
print('scale_for_pooling1', scale_for_pooling1)

print('\n')

print('scale_for_W0_x_Flatten_for_check_python', scale_for_W0_x_Flatten_for_check_python)
print('scale_for_W1_x_hd0_for_check_python', scale_for_W1_x_hd0_for_check_python)
print('scale_for_W2_x_hd1_for_check_python', scale_for_W2_x_hd1_for_check_python)

print('\n')

print('scale_for_B0_conv_for_check_python', scale_for_B0_conv_for_check_python)
print('scale_for_B1_conv_for_check_python', scale_for_B1_conv_for_check_python)

print('\n')

print('scale_for_B0_fc_for_check_python', scale_for_B0_fc_for_check_python)
print('scale_for_B1_fc_for_check_python', scale_for_B1_fc_for_check_python)
print('scale_for_B2_fc_for_check_python', scale_for_B2_fc_for_check_python)

print('\n')

print('scale_quant_bf_sigmoid_layer_for_check_python', scale_quant_bf_sigmoid_layer_for_check_python)
print('scale_dequant_bf_sigmoid_layer_for_check_python', scale_dequant_bf_sigmoid_layer_for_check_python)

print('\n')

print('max_sigmoid_layer_for_check_python', max_sigmoid_layer_for_check_python)
print('min_sigmoid_layer_for_check_python', min_sigmoid_layer_for_check_python)
print('scale_quant_sigmoid_layer_for_check_python', scale_quant_sigmoid_layer_for_check_python)
print('scale_dequant_sigmoid_layer_for_check_python', scale_dequant_sigmoid_layer_for_check_python)

scale_w0 0.03238095238095238
scale_w1 0.024761904761904763
scale_w2 0.026984126984126982


scale_b0 0.006666666666666666
scale_b1 0.006031746031746032
scale_b2 0.007619047619047619


scale_input 0.09015873015873016
scale_layer_0 0.30476190476190473
scale_layer_1 0.30476190476190473
scale_bf_sigmoid 0.7428571428571428
scale_output 0.031746031746031744


scale_calculate_convolution_0 0.007609535699423339
scale_for_pooling0 1.5257142857142858
scale_calculate_convolution_1 0.005880880880880879
scale_for_pooling1 3.126760563380282


scale_for_W0_x_Flatten 0.00957936507936508
scale_for_W1_x_hd0 0.018571428571428572
scale_for_W2_x_hd1 0.014760548093881429


scale_for_B0_conv 0.13108614232209737
scale_for_B1_conv 0.018018018018018014


scale_for_B0_fc 0.021875000000000002
scale_for_B1_fc 0.014843750000000001
scale_for_B2_fc 0.010256410256410258


scale_quant_bf_sigmoid_layer 1.3461538461538463
scale_dequant_bf_sigmoid_layer 0.7428571428571428


max_sigmoid_layer 1
min_sigmoid_layer -1
scale_qu